In [6]:
from netCDF4 import MFDataset,Dataset,num2date,date2num
import numpy as np
import datetime as dt
import xarray as xr
import os
ECCO_ts_pth='D:/ECCO/tmp/TS/'
ECCO_vel_pth='D:/ECCO/tmp/VEL/'

wnpth='J:/Reanalysis/myECCO_199201_201712_tsuvw.nc'

t_rng=['1980-01','2020-12']
My_time_ref='days since 1970-1-1 00:00:00'


In [8]:
### Read ECCO data ===============================
ts_list=list(np.sort([ECCO_ts_pth+i for i in os.listdir(ECCO_ts_pth) if i.endswith('nc')]))
# vl_list=list(np.sort([ECCO_vel_pth+i for i in os.listdir(ECCO_vel_pth) if i.endswith('nc')]))
# DATA=MFDataset(d_list[0],aggdim='time')
DATA=xr.open_mfdataset(ts_list,decode_times=False,parallel=True)
# DATA2=xr.open_mfdataset(vl_list,decode_times=False)

TIME=DATA.time.values
TIME_units=DATA.time.attrs['units']
LON,LAT=DATA.longitude.values,DATA.latitude.values
DEPTH=DATA.Z.values
tmp_dif = date2num(dt.datetime(2000,1,1),My_time_ref) - date2num(dt.datetime(2000,1,1),'days since 1992-01-01T12:00:00')
TIME_re=TIME/24+tmp_dif
ECCO_THETA=DATA.THETA.values
ECCO_SALT=DATA.SALT.values

# ECCO_EVEL=DATA2.EVEL.values
# ECCO_NVEL=DATA2.NVEL.values
# ECCO_WVEL=DATA2.WVEL.values

LAT_BNDS=DATA.latitude_bnds.values
LON_BNDS=DATA.longitude_bnds.values



In [26]:
posi_co,nega_co=np.where(LON>=0)[0],np.where(LON<0)[0]
LON_re=np.concatenate( [LON[posi_co],LON[nega_co]+360],axis=0 )
ECCO_THETA_re=np.concatenate( [ECCO_THETA[:,:,:,posi_co], ECCO_THETA[:,:,:,nega_co]] ,axis=3)
ECCO_SALT_re=np.concatenate( [ECCO_SALT[:,:,:,posi_co], ECCO_SALT[:,:,:,nega_co]] ,axis=3)


MemoryError: Unable to allocate 15.1 GiB for an array with shape (312, 50, 360, 720) and data type float32

In [13]:
def myOGCM(nc_save_name,LON,LAT,LON_bnds,LAT_bnds,DEPTH,TIME,Ref_time,values1,values2):
    
    ncfile = Dataset(nc_save_name,mode='w',format='NETCDF4')

    ncfile.createDimension('lat', len(LAT))
    ncfile.createDimension('lon', len(LON))
    ncfile.createDimension('nv', 2)
    ncfile.createDimension('depth',len(DEPTH))
    ncfile.createDimension('time',len(TIME))
    
    ncfile.title='My ECCO data '
    
    lat_bnds = ncfile.createVariable('lat_bnds', np.float32, ('lat','nv'))
    lon_bnds = ncfile.createVariable('lon_bnds', np.float32, ('lon','nv'))
    
    lat = ncfile.createVariable('lat', np.float32, ('lat',))
    lat.units = 'degrees_north'
    lon = ncfile.createVariable('lon', np.float32, ('lon',))
    lon.units = 'degrees_east'
    depth = ncfile.createVariable('depth', np.float32, ('depth',))
    depth.units = 'depth_m'
    time = ncfile.createVariable('time', np.float64, ('time',))
    time.units=Ref_time
    time.field='time, scalar, series'
    # time.cycle_length=cycle
    
    DATA1 = ncfile.createVariable('temp',np.float64,('time','depth','lat','lon'),compression='zlib') #
    DATA1.units = 'degree_C' 
    DATA1.long_name = 'ECCO temp' 
    DATA1.coordinates = "time, depth, lat, lon"
    
    DATA2 = ncfile.createVariable('salt',np.float64,('time','depth','lat','lon'),compression='zlib') 
    DATA2.units = '1e-3' 
    DATA2.long_name = 'ECCO salt' 
    DATA2.coordinates = "time, depth, lat, lon"

    # DATA3 = ncfile.createVariable('u',np.float64,('time','depth','lat','lon')) 
    # DATA3.units = 'meter second-1' 
    # DATA3.long_name = 'Eastward velocity' 
    # DATA3.coordinates = "time, depth, lat, lon"
    
    # DATA4 = ncfile.createVariable('v',np.float64,('time','depth','lat','lon')) 
    # DATA4.units = 'meter second-1' 
    # DATA4.long_name = 'Northward velocity' 
    # DATA4.coordinates = "time, depth, lat, lon"
    
    # DATA5 = ncfile.createVariable('2',np.float64,('time','depth','lat','lon')) 
    # DATA5.units = 'meter second-1' 
    # DATA5.long_name = 'Vertical velocity' #
    # DATA5.coordinates = "time, depth, lat, lon"

    # Data.field=Var.field
    lat[:] = LAT
    lon[:] = LON
    depth[:]= DEPTH
    time[:] = TIME 
    
    lat_bnds[:] = LAT_bnds
    lon_bnds[:] = LON_bnds
     
    DATA1[:] = values1
    DATA2[:] = values2
    
    ncfile.close()

In [14]:
myOGCM(wnpth,LON,LAT,LON_BNDS,LAT_BNDS,DEPTH,TIME,TIME_units,ECCO_THETA,ECCO_SALT)

In [25]:
import matplotlib.pyplot as plt

plt.pcolor(ECCO_THETA_re[0,0])

[2.5000e-01 7.5000e-01 1.2500e+00 1.7500e+00 2.2500e+00 2.7500e+00
 3.2500e+00 3.7500e+00 4.2500e+00 4.7500e+00 5.2500e+00 5.7500e+00
 6.2500e+00 6.7500e+00 7.2500e+00 7.7500e+00 8.2500e+00 8.7500e+00
 9.2500e+00 9.7500e+00 1.0250e+01 1.0750e+01 1.1250e+01 1.1750e+01
 1.2250e+01 1.2750e+01 1.3250e+01 1.3750e+01 1.4250e+01 1.4750e+01
 1.5250e+01 1.5750e+01 1.6250e+01 1.6750e+01 1.7250e+01 1.7750e+01
 1.8250e+01 1.8750e+01 1.9250e+01 1.9750e+01 2.0250e+01 2.0750e+01
 2.1250e+01 2.1750e+01 2.2250e+01 2.2750e+01 2.3250e+01 2.3750e+01
 2.4250e+01 2.4750e+01 2.5250e+01 2.5750e+01 2.6250e+01 2.6750e+01
 2.7250e+01 2.7750e+01 2.8250e+01 2.8750e+01 2.9250e+01 2.9750e+01
 3.0250e+01 3.0750e+01 3.1250e+01 3.1750e+01 3.2250e+01 3.2750e+01
 3.3250e+01 3.3750e+01 3.4250e+01 3.4750e+01 3.5250e+01 3.5750e+01
 3.6250e+01 3.6750e+01 3.7250e+01 3.7750e+01 3.8250e+01 3.8750e+01
 3.9250e+01 3.9750e+01 4.0250e+01 4.0750e+01 4.1250e+01 4.1750e+01
 4.2250e+01 4.2750e+01 4.3250e+01 4.3750e+01 4.4250e+01 4.4750

In [20]:
print(LON[posi_co][-1])
A=LON[nega_co]+360
print('ddddddddddddddd')
print(A[0])

179.75
ddddddddddddddd
180.25
